In [1]:
import pandas as pd
import numpy as np

from datetime import date

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


course_file_path = '/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/IMF/'


imf_target_values_import = pd.read_csv('/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/imf_gdp_pc_target_values.csv')
# print(imf_target_values_import)

countries = []
file_columns = {}
file_scores = {}
data_set_compile_dict = {}

file_model_df = imf_target_values_import.copy()

file_model_df = file_model_df#.iloc[1:,:]
file_model_df['Country Name'] = file_model_df['GDP per capita, current prices\n (U.S. dollars per capita)']
file_model_df.index = file_model_df['Country Name']
file_model_df=file_model_df.drop(columns = ['Country Name','GDP per capita, current prices\n (U.S. dollars per capita)'])
file_model_df.set_axis([int(float(x)) for x in file_model_df.columns],axis = 'columns', inplace=True)

country_exclusion_list = [
'Africa (Region)',
'Asia and Pacific',
'Australia and New Zealand',
'Caribbean',
'Central America',
'Central Asia and the Caucasus',
'East Asia',
'Eastern Europe ', 
'Europe',
'Middle East (Region)',
'North Africa',
'North America',
'Pacific Islands ', 
'South America',
'South Asia',
'Southeast Asia',
'Sub-Saharan Africa (Region) ',
'Western Europe',
'Western Hemisphere (Region)',
'ASEAN-5',
'Advanced economies',
'Emerging and Developing Asia',
'Emerging and Developing Europe',
'Emerging market and developing economies',
'Euro area',
'European Union',
'Latin America and the Caribbean',
'Major advanced economies (G7)',
'Middle East and Central Asia',
'Other advanced economies',
'Sub-Saharan Africa',
'World',
'nan',
'NaN',
'©IMF, 2022'
]

model_df = file_model_df.copy()
model_df = model_df[~model_df.index.isin(country_exclusion_list)]
model_df=model_df.fillna(0)
model_df = model_df.loc[model_df.index.notna()]

train_model_df = model_df.copy()
test_model_df = model_df.copy()

data_model_file_path = '/Users/stephenontko/Documents/University of Michigan/UMSI/UMSI MADS/UMSI MADS 2023/SIADS 699 Capstone/Data/'

metadata_df = pd.read_csv(course_file_path+'/file_metadata_df.csv')


In [2]:

def process_data_for_model(train_year, inputdate):
    print('training year: ',train_year, ' date: ',inputdate)

    train_y=train_model_df[[train_year+1]].replace('no data',0).fillna(0)
    test_y=test_model_df[[train_year+2]].replace('no data',0).fillna(0)

    exclude_list = [ 
        'Eastern Caribbean Currency Union (ECCU)',
        'European Central Bank'
    ]
    train_X = pd.read_csv(course_file_path+'train_feature_data_'+str(train_year)+'_'+str(inputdate)+'.csv')
    train_X.index = train_X['Country Name']
    train_X = train_X.drop(columns =['Country Name'])
    train_X = train_X[~train_X.index.isin(exclude_list)]

    test_X = pd.read_csv(course_file_path+'test_feature_data_'+str(train_year+1)+'_'+str(inputdate)+'.csv')
    test_X.index = test_X['Country Name']
    test_X = test_X.drop(columns =['Country Name'])
    test_X = test_X[~test_X.index.isin(exclude_list)]

    code_names = train_X.columns
    filename = None

    train_data_to_fit_model = train_y.merge(train_X  , left_on='Country Name', right_on='Country Name', how='inner')#.reset_index()


    test_data_to_fit_model = test_y.merge(test_X  , left_on='Country Name', right_on='Country Name', how='inner')#.reset_index()


    train_df = train_data_to_fit_model.iloc[:,:]
    test_df = test_data_to_fit_model.iloc[:,:]


    train_X = train_df.drop(columns =[train_year+1])
    train_y = train_df[[train_year+1]]

    test_X = test_df.drop(columns =[train_year+2])
    test_y = test_df[[train_year+2]]

    exclude_column_list = ['Xgdppc','PPPGDP']#,'G_X_G01_GDP_PT','G_XWDG_G01_GDP_PT',]

    for train_column in train_X.columns:
        if train_column in exclude_column_list:
            train_X = train_X.drop(columns =[train_column])
    for test_column in test_X.columns:
        if test_column in exclude_column_list:
            test_X = test_X.drop(columns =[test_column])

    feature_list = [x for x in train_X.columns]
    feature_list = [x for x in feature_list if x in test_X.columns]

    train_X = train_X[feature_list]
    test_X = test_X[feature_list]

    return train_X, code_names#train_y, test_X, test_y


# train_year = 2021
# filedate = '2023-03-12'

train_year = 2020
filedate = '2023-04-04'

# train_X, train_y, test_X, test_y, code_names, filename =  process_data_for_model(train_year, filedate)
train_X, code_names =  process_data_for_model(train_year, filedate)

def calc_variance_inflation_factor(train_X):
    train_X = train_X.dropna()
    train_X = add_constant(train_X)
    train_X_scaled = StandardScaler().fit(train_X).transform(train_X)
    train_X_scaled = add_constant(train_X_scaled)

    vif = pd.DataFrame()
    vif['features'] = train_X.columns
    vif['feature_sums'] = [x for x in train_X.sum(axis=0)]
    vif = vif[vif['feature_sums']!=0]
    train_X = train_X[vif['features']]

    vif['VIF'] = [variance_inflation_factor(train_X, i) for i in range(train_X.shape[1])]

    print(vif['VIF'])
    return vif

vif_df = calc_variance_inflation_factor(train_X)
vif_df = vif_df.sort_values(by=['VIF'], ascending=False).round(4)#df.sort_values(by=['col1'])

print(vif_df)

# vif_df.to_csv(data_model_file_path+'vif_train_model_df_'+str(train_year)+'_'+str(filedate)+'.csv', index=True)

def calc_corr(train_X):
    print(train_X)
    train_X.dropna()

    corr_df = pd.DataFrame()
    corr_df['features'] = train_X.columns
    corr_df['feature_sums'] = [x for x in train_X.sum(axis=0)]
    corr_df = corr_df[corr_df['feature_sums']!=0]
    train_X = train_X[corr_df['features']]

    print(abs(train_X.corr()))
    corr_df = abs(train_X.corr())
    corr_results = corr_df.sum(axis = 0)
    print(corr_results)
    print(corr_results.shape)
    print(corr_results.values)
    print(corr_results.index)
    # corr_results.to_csv(data_model_file_path+'corr_train_model_df_'+str(train_year)+'_'+str(filedate)+'.csv', index=True).sort_values(by=['VIF'], ascending=False).round(4)#

# calc_corr(train_X)



training year:  2020  date:  2023-04-04
                    10RA__XDC  11____EUR     11____USD     11____XDC  \
Country Name                                                           
Afghanistan               0.0        0.0  0.000000e+00  2.491308e+10   
Albania                   0.0        0.0  0.000000e+00  1.092416e+11   
Algeria                   0.0        0.0  0.000000e+00  8.768488e+11   
Andorra                   0.0        0.0  0.000000e+00  0.000000e+00   
Angola                    0.0        0.0  0.000000e+00  8.228547e+11   
...                       ...        ...           ...           ...   
Vietnam                   0.0        0.0  0.000000e+00  4.390028e+14   
West Bank and Gaza        0.0        0.0  3.730317e+08  0.000000e+00   
Yemen                     0.0        0.0  0.000000e+00  6.939473e+11   
Zambia                    0.0        0.0  0.000000e+00  4.693929e+11   
Zimbabwe                  0.0        0.0  0.000000e+00  1.198580e+13   

                    12A

                    const  10RA__XDC  11____EUR     11____USD     11____XDC  \
Country Name                                                                  
Afghanistan           1.0        0.0        0.0  0.000000e+00  2.491308e+10   
Albania               1.0        0.0        0.0  0.000000e+00  1.092416e+11   
Algeria               1.0        0.0        0.0  0.000000e+00  8.768488e+11   
Andorra               1.0        0.0        0.0  0.000000e+00  0.000000e+00   
Angola                1.0        0.0        0.0  0.000000e+00  8.228547e+11   
...                   ...        ...        ...           ...           ...   
Vietnam               1.0        0.0        0.0  0.000000e+00  4.390028e+14   
West Bank and Gaza    1.0        0.0        0.0  3.730317e+08  0.000000e+00   
Yemen                 1.0        0.0        0.0  0.000000e+00  6.939473e+11   
Zambia                1.0        0.0        0.0  0.000000e+00  4.693929e+11   
Zimbabwe              1.0        0.0        0.0  0.0

/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/stephenontko/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


KeyboardInterrupt: 